### Save Integrated Gradients

After running `transformer.ipynb`, you should have a model saved at the checkpoint path below. The first part of this notebook just reads in the data into a data loader, teh second part runs `IntegratedGradients` from the captum package and then saves the output into a csv for visualization within R.

In [4]:
input_path = "../data/blooms.csv"
checkpoint = "lightning_logs/version_0/checkpoints/epoch=69-step=1680.ckpt"
output_path = "../data/attributions.csv"
device = "cpu"

This reads in the data. We've increased the dataloader size because we aren't training, so don't need to be so stingy with GPU memory.

In [5]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from transformer import LinearData, Transformer, LitTransformer

samples_df = pd.read_csv(input_path)
dataset = LinearData(samples_df)
loader = DataLoader(dataset, batch_size=64)

This runs the attribution and saves it to the `output_path` defined above.

In [6]:
from captum.attr import IntegratedGradients

lit_model = LitTransformer.load_from_checkpoint(checkpoint, model = Transformer()).to(device)

def forward(model):
  return lambda x: model(x)[1]

ig = IntegratedGradients(forward(lit_model.model))
x, y = next(iter(loader))
i_hat = ig.attribute(x.to(device))
np.savetxt(output_path, np.reshape(i_hat, (64, -1)).cpu(), delimiter=',')